In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y #integrate zenml with sklean
%pip install pyparsing==2.4.2 #required for colab
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True) #atuo restart kernal

NumExpr defaulting to 2 threads.
⠙ Installing integrations...


{'status': 'ok', 'restart': True}

In [ ]:
NGROCK_TOKEN ='2lmBGh70bemsJxcwr7TufMMzfqF_3nBkSpPGvUZzJoyiWjsw' #

In [ ]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROCK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ZEN ML ***SETUP***

In [ ]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠹ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
infor

**This is an example experimentation code for ml model**




In [ ]:
from string import digits
import pandas as pd
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split



def train_test() -> None:'''train and test the sklearn svc classifier on digits'''

digits = load_digits()
data = digits.images.reshape((len(digits.images),-1))
x_train,x_test,y_train,y_test = train_test_split(data,digits.target,random_state=42,test_size=0.2,shuffle=False)
model = SVC(gamma =0.001)
model.fit(x_train,y_train)
test_acc = model.score(x_test,y_test)
print(f"the accuracy is :{test_acc}")

train_test()

the accuracy is :0.9583333333333334



**Turning  the same code into ML pipelines with ZenML**

In [ ]:
from zenml import step, pipeline
from typing_extensions import Annotated
from typing import Tuple
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC

@step
def importer() -> Tuple[Annotated[np.ndarray, 'x_train'],
                        Annotated[np.ndarray, 'x_test'],
                        Annotated[np.ndarray, 'y_train'],
                        Annotated[np.ndarray, 'y_test']]:
    """Load the digits dataset and split it into training and testing sets."""
    # Load the digits dataset
    digits = load_digits()
    # Flatten the images
    data = digits.images.reshape(len(digits.images), -1)
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(
        data, digits.target, random_state=42, test_size=0.2, shuffle=False
    )
    # Return the split data
    return x_train, x_test, y_train, y_test

@step
def svc_trainer(
    x_train: np.ndarray,
    y_train: np.ndarray
) -> ClassifierMixin:
    """Train an SVC model."""
    model = SVC(gamma=0.001)
    model.fit(x_train, y_train)
    return model

@step
def svc_eval(
    x_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin
) -> float:
    """Evaluate the model accuracy."""
    score = model.score(x_test, y_test)
    print(f'The test accuracy is: {score}')
    return score



**CREATING ML PIPELINE**


In [ ]:
@pipeline
def digits_pipeline() -> float:
    """Link all steps together into a pipeline."""
    x_train, x_test, y_train, y_test = importer()
    model = svc_trainer(x_train=x_train, y_train=y_train)
    score = svc_eval(x_test=x_test, y_test=y_test, model=model)
    return score


**Running Zenml Pipeline**

In [ ]:
if __name__ == "__main__":
    result = digits_pipeline()
    print(f'Pipeline result: {result}')

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of svc_eval.
Step svc_eval has started.
Pipeline run has finished in 1.627s.
Pipeline result: body=PipelineRunResponseBody(created=datetime.datetime(2024, 9, 8, 7, 40, 8, 929522), updated=datetime.datetime(2024, 9, 8, 7, 40, 10, 589966), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 9, 8, 6, 7, 31, 212484), updated=datetime.datetime(2024, 9, 8, 6, 7, 31, 212490), active=True, activation_token=None, full_name='', email_opted_in=None, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('53f3380a-1248-46bf-bb0c-40f9eccb4ea7'), permission_denied=False, name='default'), st

**visualize the pipeline**

In [ ]:
from zenml.environment import Environment
from pyngrok import ngrok

def zenml_dashboard(port=8239):
    if Environment.in_google_colab():
        # Start ngrok to expose the local port
        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")

        # Start ZenML in the background
        !zenml up --blocking --port {port}
    else:
        # Start ZenML normally if not in Colab
        !zenml up --port {port}

    # This function should start or interact with the ZenML dashboard if needed
    # If you don't have this function, you may omit it
    # start_zenml_dashboard()

# Call the function to execute
zenml_dashboard()




INFO:pyngrok.ngrok:Opening tunnel named: http-8239-2cc7fc6c-1cee-44cb-ae0e-36cfccae363f


Opening tunnel named: http-8239-2cc7fc6c-1cee-44cb-ae0e-36cfccae363f


INFO:pyngrok.process.ngrok:t=2024-09-08T07:40:14+0000 lvl=info msg="no configuration paths supplied"


t=2024-09-08T07:40:14+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-09-08T07:40:14+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

In Colab, use this URL instead: NgrokTunnel: "https://a545-35-197-10-217.ngrok-free.app" -> "http://localhost:8239"!t=2024-09-08T07:40:15+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8239-2cc7fc6c-1cee-44cb-ae0e-36cfccae363f addr=http://localhost:8239 url=https://a545-35-197-10-217.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-09-08T07:40:15+0000 lvl=info msg=end pg=/api/tunnels id=8e482f0912567bde status=201 dur=63.447279ms



t=2024-09-08T07:40:15+0000 lvl=info msg=end pg=/api/tunnels id=8e482f0912567bde status=201 dur=63.447279ms
NumExpr defaulting to 2 threads.
The local ZenML dashboard is about to deploy in a blocking process. You can connect to it using the 
'default' username and an empty password.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [28286]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8239 (Press CTRL+C to quit)


INFO:pyngrok.process.ngrok:t=2024-09-08T07:41:56+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2024-09-08T07:41:56+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:     Finished server process [28286]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 474, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 68, in serve
    with self.capture_signals():
  File "/usr/lib/python3.10/contextlib.py", line 142, in __exit__
    next(